In [1]:
import sys; sys.path.append('../3rdparty/ElasticRods/python')
import elastic_rods, elastic_knots
import numpy as np, matplotlib.pyplot as plt, time, io, os

from helpers import *
from parametric_curves import *
import py_newton_optimizer

from linkage_vis import LinkageViewer as Viewer, CenterlineViewer
from tri_mesh_viewer import PointCloudViewer, PointCloudMesh

%load_ext autoreload
%autoreload 2
import time
import parallelism
parallelism.set_max_num_tbb_threads(1)
energy = []

In [2]:
#only doubles the nodes
def add_n_nodes(n,nodes): 
    nodes = nodes.reshape(-1,3)
    diff = np.concatenate([nodes[:-1] - nodes[1:], [nodes[-1] - nodes[0]]])
    inbetween = nodes- (diff/2)
    result = np.repeat(nodes,2,axis=0)
    result[1::2] = inbetween
    return result

In [3]:
def move_linear(rods, target,steps=100):
    diff = rods.getDoFs()[:1200] - target
    DoFs = rods.getDoFs()
    path = [DoFs.copy()]
    for i in range(int(steps)):
        DoFs[:1200] -= 1/steps *diff 
        rods.setDoFs(DoFs)
        path.append(DoFs.copy())
        view.update()
    return path

In [4]:
# ...or generate a parametric knot
p, q, a, r = 2, 3, 2, 4
rod_radius = 0.2

material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])
centerline = generate_curve(torus_knot(p=p, q=q, a=a, r=r), npts=400)
centerline = generate_curve(figure_eight(r=r), npts=400)
print(len(centerline))
pr = define_periodic_rod(centerline, material)
rod_list = elastic_knots.PeriodicRodList([pr])
A_DoFs = []

400


In [5]:
view = Viewer(rod_list, width=1024, height=800)
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.28, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0),…

In [21]:
def callback(problem, iteration):
    if iteration % 5 == 0:
        A_DoFs.insert(0,rod_list.getDoFs())
        view.update()
        
optimizerOptions = py_newton_optimizer.NewtonOptimizerOptions()
optimizerOptions.niter = 1000
optimizerOptions.gradTol = 1e-8
hessianShift = 1e-4 * compute_min_eigenval_straight_rod(pr)

problemOptions = elastic_knots.ContactProblemOptions()
problemOptions.contactStiffness = 1e3
problemOptions.dHat = 2*rod_radius

fixedVars = []   # all the degrees of freedom can be modified by the optimizer

In [22]:
# Optimize
def optimize():
    report = elastic_knots.compute_equilibrium(
        rod_list, problemOptions, optimizerOptions, 
        fixedVars=fixedVars,
        externalForces=np.zeros(rod_list.numDoF()),
        softConstraints=[],
        callback=callback,
        hessianShift=hessianShift
        )
    view.update()

In [23]:
def go_backwards():
    for dof in A_DoFs:
        rod_list.setDoFs(dof)
        time.sleep(0.1)
        view.update()

In [24]:
file = '../data/L400-r0.2-UpTo9Crossings/4_1/0033.obj'
optimize()
time.sleep(3)
go_backwards()
time.sleep(5)
nodes = read_nodes_from_file(file).flatten() /2.5
path=move_linear(rod_list,nodes)
time.sleep(1)
optimize()
A_DoFs=[]

The simulations will continue, but the result might be non-physical due to spurious contact forces between neighboring edges.
Consider decreasing the cross-section radius or using a coarser polyline.
Increasing the minContactEdgeDist parameter would remove non-physical contact forces, too, but only at the expense of topology preservation guarantees.



0	4.21866	2.92813	2.92813	0.25	1
1	3.77106	2.37009	2.37009	1	0
2	3.246	0.905219	0.905219	1	0
3	3.14987	1.45537	1.45537	1	1
4	2.93223	0.329689	0.329689	0.25	0
5	2.89339	0.417117	0.417117	1	0
6	2.8289	0.601675	0.601675	1	0
7	2.78723	0.598681	0.598681	1	0
8	2.74874	0.670258	0.670258	0.5	0
9	2.71805	0.465385	0.465385	1	0
10	2.67611	0.139337	0.139337	0.140625	0
11	2.66965	0.137828	0.137828	1	0
12	2.661	0.369746	0.369746	1	0
13	2.63475	0.0529302	0.0529302	0.25	1
14	2.62966	0.065974	0.065974	0.0625	0
15	2.62821	0.0635042	0.0635042	0.0234375	0
16	2.62767	0.062251	0.062251	0.015625	0
17	2.62739	0.166331	0.166331	1	1
18	2.62615	0.036812	0.036812	1	1
19	2.62529	0.00908782	0.00908782	1	1
20	2.6241	0.00666133	0.00666133	1	1
21	2.62226	0.0133965	0.0133965	1	1
22	2.61948	0.0471311	0.0471311	1	1
23	2.61633	0.1491	0.1491	1	1
24	2.6158	0.0392736	0.0392736	1	1
25	2.61551	0.0100551	0.0100551	1	1
26	2.615	0.00346464	0.00346464	1	1
27	2.61407	0.00680283	0.00680283	1	1
28	2.61256	0.0324983	0.0324983	1	1
29	2

The simulations will continue, but the result might be non-physical due to spurious contact forces between neighboring edges.
Consider decreasing the cross-section radius or using a coarser polyline.
Increasing the minContactEdgeDist parameter would remove non-physical contact forces, too, but only at the expense of topology preservation guarantees.



0	5662.16	4904.69	4904.69	1	1
1	5162.75	3301.92	3301.92	1	1
2	4733.93	1913.42	1913.42	1	1
3	4345.47	1003.76	1003.76	1	1
4	3877.48	548.465	548.465	1	1
5	3257.94	337.903	337.903	1	1
6	2571.88	221.766	221.766	1	1
7	2347.7	189.285	189.285	1	1
8	2081.99	161.153	161.153	1	1
9	1990.84	153.04	153.04	1	1
10	1873.34	149.464	149.464	1	1
11	1740.54	132.437	132.437	1	1
12	1706.64	128.019	128.019	1	1
13	1653.65	132.964	132.964	1	1
14	1629.71	135.217	135.217	1	1
15	1582.81	137.473	137.473	1	1
16	1561.7	134.278	134.278	1	1
17	1522.7	127.813	127.813	1	1
18	1504.89	125.279	125.279	1	1
19	1469.05	123.58	123.58	1	1
20	1452.84	120.189	120.189	1	1
21	1421.27	118.011	118.011	1	1
22	1406.06	118.404	118.404	1	1
23	1373.16	120.694	120.694	1	1
24	1358.25	116.766	116.766	1	1
25	1331.17	107.47	107.47	1	1
26	1319.02	103.815	103.815	1	1
27	1296.09	98.9337	98.9337	1	1
28	1261.41	180.639	180.639	1	1
29	1231.87	110.288	110.288	1	1
30	1190.58	93.2668	93.2668	1	1
31	1171.79	91.1868	91.1868	1	1
32	1134.43	90.2642	90.2642	

In [10]:
view.update()

In [11]:
#Same Knot: 4_1: L100, L200, L400, L500
#Different L300

In [12]:
#TODO: zwei minima verbinden und minimieren -> entsteht ein Kreis?
# lineare Transformation mit minimieren kombinieren (loop: linear, min)
# transformation to '../data/4_1-smooth.obj' and move linear towards different minmum -> will it go to it?